# Mixed Symbolic-Numeric Perturbation Theory

Perturbation methods are a collection of techniques to solve hard problems that generally don't have a closed solution, but depend on a tunable parameter and have closed or easy solutions for some values of this parameter. The main idea is to assume a solution that is a power series in the tunable parameter (say $ϵ$), such that $ϵ = 0$ corresponds to an easy solution, and then solve iteratively for higher-order corrections.

This tutorial shows how to mix symbolic manipulations and numerical methods to solve algebraic equations with perturbation theory.

## Reminder: Taylor Series

Say we want to approximate $\cos(x)$ with a polynomial around $x_0=0$

$$ P(x) = c_0 + c_1 x + c_2 x^2. $$

We want to match value and the derivatives at that point.

So 

$$ c_0 = \cos(0) = 1. $$

Then 
$$ \frac{d(\cos(x))}{dx}(0) = -\sin(0) = 0 $$
$$ \frac{dP}{dx}(0) = c_1 + 2 c_2 0, $$
hence set $c_1=1$ to match the derivative.

Same game to find $c_2 = (-\frac{1}{2}).$

General:

$$ P(x) = f(a) + \frac{df}{dx}(a) \frac{(x-a)^1}{1!} + \frac{d^2f}{dx^2}(a) \frac{(x-a)^2}{2!} + ... $$

## Solving the Quintic Equation <a id="solving_the_quintic_equation" />

The “hello world!” analog of perturbation problems is to find a real solution $x$ to the quintic (fifth-order) equation

In [4]:
using Symbolics
@variables x
quintic = x^5 + x ~ 1

x + x^5 ~ 1

According to Abel's theorem, a general quintic equation does not have a closed form solution. But we can easily solve it numerically using Newton's method (here implemented for simplicity, and not performance):

In [6]:
function my_newton(eq, x, x0, abstol=1e-8, maxiter=50)
    f = eq.lhs - eq.rhs
    f_dash = Symbolics.derivative(f, x)

    x_old = x0

    for i = 1:maxiter
        x_new = substitute(x - f / f_dash, x => x_old)
        if abs(x_new - x_old) < abstol
            return x_new
        else
            x_old = x_new
        end
    i += 1
    end
    error("Newton's method failed to converge.")
end

x_newton = my_newton(quintic, x, 1.0)
println("Newton's method solution: x = ", x_newton)

Newton's method solution: x = 0.7548776662466927


To solve the problem with perturbation theory, we must introduce an expansion variable $\epsilon$ in the equation:

In [7]:
@variables ϵ        # expansion variable
quintic = x^5 + ϵ*x ~ 1

x*ϵ + x^5 ~ 1

If $ϵ = 1$, we get our original problem. With $ϵ = 0$, the problem transforms to the easy quintic equation $x^5 = 1$ with the trivial real solution $x = 1$ (and four complex solutions which we ignore). 

Next, expand $x$ as a power series in $ϵ$:

In [8]:
x_coeffs, = @variables a[0:7]       # create Taylor series coefficients
x_taylor = series(x_coeffs, ϵ)      # expand x in a power series in ϵ

a[0] + a[1]*ϵ + a[2]*(ϵ^2) + a[3]*(ϵ^3) + a[4]*(ϵ^4) + a[5]*(ϵ^5) + a[6]*(ϵ^6) + a[7]*(ϵ^7)

Then insert this into the quintic equation and expand it, too, to the same order:

In [9]:
quintic_taylor = substitute(quintic, x => x_taylor)
quintic_taylor = taylor(quintic_taylor, ϵ, 0:7)

a[0]^5 + (a[0] + 5(a[0]^4)*a[1])*ϵ + (1//2)*(2a[1] + 10(a[0]^4)*a[2] + 20(a[0]^3)*(a[1]^2))*(ϵ^2) + (1//6)*(6a[2] + 30(a[0]^4)*a[3] + 120(a[0]^3)*a[1]*a[2] + 60(a[0]^2)*(a[1]^3))*(ϵ^3) + (1//24)*(24a[3] + 120(a[0]^4)*a[4] + 480(a[0]^3)*a[1]*a[3] + (240//1)*(a[0]^3)*(a[2]^2) + 720(a[0]^2)*(a[1]^2)*a[2] + 120a[0]*(a[1]^4))*(ϵ^4) + (1//120)*(120a[4] + 600(a[0]^4)*a[5] + 2400(a[0]^3)*a[1]*a[4] + 2400(a[0]^3)*a[2]*a[3] + 3600(a[0]^2)*(a[1]^2)*a[3] + (3600//1)*(a[0]^2)*a[1]*(a[2]^2) + 2400a[0]*(a[1]^3)*a[2] + 120(a[1]^5))*(ϵ^5) + (1//720)*(720a[5] + 3600(a[0]^4)*a[6] + 14400(a[0]^3)*a[1]*a[5] + 14400(a[0]^3)*a[2]*a[4] + (7200//1)*(a[0]^3)*(a[3]^2) + 21600(a[0]^2)*(a[1]^2)*a[4] + 43200(a[0]^2)*a[1]*a[2]*a[3] + (7200//1)*(a[0]^2)*(a[2]^3) + 14400a[0]*(a[1]^3)*a[3] + (21600//1)*a[0]*(a[1]^2)*(a[2]^2) + 3600(a[1]^4)*a[2])*(ϵ^6) + (1//5040)*(5040a[6] + 25200(a[0]^4)*a[7] + 100800(a[0]^3)*a[1]*a[6] + 100800(a[0]^3)*a[2]*a[5] + 100800(a[0]^3)*a[3]*a[4] + 151200(a[0]^2)*(a[1]^2)*a[5] + 302400(a[0]^2

This messy equation must hold for each power of $ϵ$, so we can separate it into one nicer equation per order:

In [10]:
quintic_eqs = taylor_coeff(quintic_taylor, ϵ, 0:7)
quintic_eqs[1:5] # for readability, show only 5 shortest equations

5-element Vector{Equation}:
 a[0]^5 ~ 1//1
 a[0] + 5(a[0]^4)*a[1] ~ 0//1
 (1//2)*(2a[1] + 10(a[0]^4)*a[2] + 20(a[0]^3)*(a[1]^2)) ~ 0//1
 (1//6)*(6a[2] + 30(a[0]^4)*a[3] + 120(a[0]^3)*a[1]*a[2] + 60(a[0]^2)*(a[1]^3)) ~ 0//1
 (1//24)*(24a[3] + 120(a[0]^4)*a[4] + 480(a[0]^3)*a[1]*a[3] + (240//1)*(a[0]^3)*(a[2]^2) + 720(a[0]^2)*(a[1]^2)*a[2] + 120a[0]*(a[1]^4)) ~ 0//1